In [29]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shivam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
df1 = pd.read_csv ('C:/Users/Shivam/Twitter/output.csv')
#selecting only the tweets which have been categorised as offensive by our model
df = df.loc[df["Prediction"] ==1 ]
#print(df)
print(len(df. index))

63


In [37]:
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RTs@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [42]:
df['clean_tweet'] = df.tweet.apply(clean_tweet)
#print(df.head())

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

<>:4: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
C:\Users\Shivam\AppData\Local\Temp/ipykernel_14096/3813916862.py:4: DeprecationWarning: invalid escape sequence \w
  vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')
C:\Users\Shivam\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [40]:
# print(tf_feature_names)
# print(tf.shape)
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)
model.fit(tf)


LatentDirichletAllocation(random_state=0)

In [28]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)
no_top_words = 10
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,’t,60.5,politician,70.5,amp,100.0,politician,198.7,politician,62.4,polit,92.9,politician,76.4,’,121.0,polit,319.4,politician,51.5
1,i,52.1,right,46.1,polit,67.8,need,43.4,stop,54.1,make,43.6,would,71.2,it,71.0,peopl,54.8,call,37.6
2,live,41.9,good,32.8,come,39.1,know,34.5,chang,33.1,care,34.1,peopl,55.4,one,66.1,get,38.3,law,28.1
3,don,41.2,much,31.1,state,32.1,support,28.1,power,29.1,american,30.6,vote,48.2,politician,40.1,go,30.4,’,26.3
4,countri,40.1,money,28.0,even,26.6,go,24.2,also,27.9,sport,30.3,trump,41.0,polit,37.7,person,26.4,day,23.1
5,think,38.2,peopl,27.5,today,26.1,republican,21.1,like,26.5,play,29.1,take,28.4,thing,24.3,us,25.1,way,18.3
6,’m,28.1,media,25.6,parti,25.5,start,19.5,’t,19.1,talk,24.3,presid,27.7,time,20.4,see,23.2,work,17.9
7,politician,26.7,like,21.4,know,24.9,think,19.5,need,13.5,peopl,16.6,work,20.9,mani,18.0,trump,20.1,democrat,15.8
8,polit,24.3,us,20.9,like,12.0,time,17.6,get,12.4,still,13.7,us,20.8,say,17.3,want,18.2,let,15.2
9,life,18.1,govern,18.9,make,7.5,year,16.5,want,10.0,time,12.2,get,19.5,’t,14.2,keep,15.9,job,12.3
